In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

**ml-100k Preprocess**

In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hybrid reccomendation using ALS and KNN/datasets/ml-latest-small/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hybrid reccomendation using ALS and KNN/datasets/ml-latest-small/ratings.csv')
tags = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hybrid reccomendation using ALS and KNN/datasets/ml-latest-small/tags.csv')
print('movies: ', movies.shape)
print('ratings: ', ratings.shape)
print('tags: ', tags.shape)

movies:  (9742, 3)
ratings:  (100836, 4)
tags:  (3683, 4)


In [ ]:
movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
df = pd.merge(ratings, movies, on='movieId' , how='left')
df = df.drop('title', axis=1)
df.head(5)

,userId,movieId,rating,timestamp,genres
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Comedy|Romance
2,1,6,4.0,964982224,Action|Crime|Thriller
3,1,47,5.0,964983815,Mystery|Thriller
4,1,50,5.0,964982931,Crime|Mystery|Thriller


In [ ]:
df['genres'] = df['genres'].str.split('|')
df.head()

,userId,movieId,rating,timestamp,genres
0,1,1,4.0,964982703,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,1,3,4.0,964981247,"[Comedy, Romance]"
2,1,6,4.0,964982224,"[Action, Crime, Thriller]"
3,1,47,5.0,964983815,"[Mystery, Thriller]"
4,1,50,5.0,964982931,"[Crime, Mystery, Thriller]"


In [ ]:
tags['tag'] = tags['tag'].str.split('|')
tags.drop('timestamp', axis=1, inplace=True)
tags = tags.groupby(['userId','movieId'])['tag'].apply(lambda x: ','.join(x.astype(str))).reset_index()
tags.head()

,userId,movieId,tag
0,2,60756,"['funny'],['Highly quotable'],['will ferrell']"
1,2,89774,"['Boxing story'],['MMA'],['Tom Hardy']"
2,2,106782,"['drugs'],['Leonardo DiCaprio'],['Martin Scors..."
3,7,48516,['way too long']
4,18,431,"['Al Pacino'],['gangster'],['mafia']"


In [ ]:
df = pd.merge(df, tags, on=['userId','movieId'], how='left')

In [ ]:
df.head()

,userId,movieId,rating,timestamp,genres,tag
0,1,1,4.0,964982703,"[Adventure, Animation, Children, Comedy, Fantasy]",NaN
1,1,3,4.0,964981247,"[Comedy, Romance]",NaN
2,1,6,4.0,964982224,"[Action, Crime, Thriller]",NaN
3,1,47,5.0,964983815,"[Mystery, Thriller]",NaN
4,1,50,5.0,964982931,"[Crime, Mystery, Thriller]",NaN


In [ ]:
df['tag'] = df['tag'].apply(lambda d: d if isinstance(d, list) else [])
df['genres'] = df['genres'].apply(lambda d: d if isinstance(d, list) else [])
df.head()
                                  

,userId,movieId,rating,timestamp,genres,tag
0,1,1,4.0,964982703,"[Adventure, Animation, Children, Comedy, Fantasy]",[]
1,1,3,4.0,964981247,"[Comedy, Romance]",[]
2,1,6,4.0,964982224,"[Action, Crime, Thriller]",[]
3,1,47,5.0,964983815,"[Mystery, Thriller]",[]
4,1,50,5.0,964982931,"[Crime, Mystery, Thriller]",[]


In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2, stratify=df.userId) #stratify google search

In [ ]:
train_data = train_data.sort_values(['userId', 'movieId'])
train_data.head()

,userId,movieId,rating,timestamp,genres,tag
0,1,1,4.0,964982703,"[Adventure, Animation, Children, Comedy, Fantasy]",[]
1,1,3,4.0,964981247,"[Comedy, Romance]",[]
2,1,6,4.0,964982224,"[Action, Crime, Thriller]",[]
3,1,47,5.0,964983815,"[Mystery, Thriller]",[]
4,1,50,5.0,964982931,"[Crime, Mystery, Thriller]",[]


In [ ]:
test_data = test_data.sort_values(['userId','movieId'])
test_data.head()

,userId,movieId,rating,timestamp,genres,tag
8,1,151,5.0,964984041,"[Action, Drama, Romance, War]",[]
11,1,216,5.0,964981208,[Comedy],[]
12,1,223,3.0,964980985,[Comedy],[]
15,1,260,5.0,964981680,"[Action, Adventure, Sci-Fi]",[]
16,1,296,3.0,964982967,"[Comedy, Crime, Drama, Thriller]",[]


In [ ]:
print("shape of training data",train_data.shape[0])
print("shape of testing data",test_data.shape[0])


shape of training data 80668
shape of testing data 20168


In [ ]:
# train_data.to_csv('/content/drive/MyDrive/Colab Notebooks/Hybrid reccomendation using ALS and KNN/datasets/created/training_data.csv', index = False)
# test_data.to_csv('/content/drive/MyDrive/Colab Notebooks/Hybrid reccomendation using ALS and KNN/datasets/created/testing_data.csv', index = False)

**Ml-1 million preprocess**

In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hybrid reccomendation using ALS and KNN/datasets/ml-1m/movies.dat',sep='::',header=None,names=["movieId", "title", "genres"])
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hybrid reccomendation using ALS and KNN/datasets/ml-1m/ratings.dat',sep='::',header=None,names=["userId", "movieId", "rating", "timestamp"])
tags = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hybrid reccomendation using ALS and KNN/datasets/ml-1m/users.dat',sep='::',header=None,names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])
print('movies: ', movies.shape)
print('ratings: ', ratings.shape)
print('tags: ', tags.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


movies:  (3883, 3)
ratings:  (1000209, 4)
tags:  (6040, 5)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
tags.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
df = pd.merge(ratings, movies, on='movieId' , how='left')
df = df.drop('title', axis=1)
df.head(5)

,userId,movieId,rating,timestamp,genres
0,1,1193,5,978300760,Drama
1,1,661,3,978302109,Animation|Children's|Musical
2,1,914,3,978301968,Musical|Romance
3,1,3408,4,978300275,Drama
4,1,2355,5,978824291,Animation|Children's|Comedy


In [ ]:
df['genres'] = df['genres'].str.split('|')
df.head()

,userId,movieId,rating,timestamp,genres
0,1,1193,5,978300760,[Drama]
1,1,661,3,978302109,"[Animation, Children's, Musical]"
2,1,914,3,978301968,"[Musical, Romance]"
3,1,3408,4,978300275,[Drama]
4,1,2355,5,978824291,"[Animation, Children's, Comedy]"


In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2, stratify=df.userId)

In [ ]:
train_data = train_data.sort_values(['userId', 'movieId'])
train_data.head()

,userId,movieId,rating,timestamp,genres
40,1,1,5,978824268,"[Animation, Children's, Comedy]"
25,1,48,5,978824351,"[Animation, Children's, Musical, Romance]"
39,1,150,5,978301777,[Drama]
44,1,260,4,978300760,"[Action, Adventure, Fantasy, Sci-Fi]"
23,1,527,5,978824195,"[Drama, War]"


In [ ]:
test_data = test_data.sort_values(['userId','movieId'])
test_data.head()

,userId,movieId,rating,timestamp,genres
10,1,595,5,978824268,"[Animation, Children's, Musical]"
35,1,783,4,978824291,"[Animation, Children's, Musical]"
37,1,1022,5,978300055,"[Animation, Children's, Musical]"
22,1,1270,5,978300055,"[Comedy, Sci-Fi]"
6,1,1287,5,978302039,"[Action, Adventure, Drama]"


In [ ]:
print("shape of training data",train_data.shape[0])
print("shape of testing data",test_data.shape[0])


shape of training data 800167
shape of testing data 200042


In [ ]:
train_data.to_csv('/content/drive/MyDrive/Colab Notebooks/Hybrid reccomendation using ALS and KNN/datasets/created/1m_training_data.csv', index = False)
test_data.to_csv('/content/drive/MyDrive/Colab Notebooks/Hybrid reccomendation using ALS and KNN/datasets/created/1m_testing_data.csv', index = False)